In [41]:
"""
Generic Implementation of a Transition-Assigned Finite State 
Machine (Mealy Automaton)
"""
class TSAFM:
    """
    where:
    Q - Set of states
    S - input alphabet
    R - output alphabet
    f - state transition function (Q x S -> Q)
    g - output function (Q x S -> R)
    qi - initial state
    """
    def __init__(self, Q, S, R, f, g, qi):
        self.Q = Q
        self.S = S
        self.R = R
        self.f = f
        self.g = g
        self.qi = qi

    def run(self, str):
        q = self.qi
        # str = str.split("")[::-1]
        # str = str[::-1]
        while len(str) > 1:
            # print(str.pop())
            # Reimplement
            q = self.f[(q, str[0])]
            str = str[1:]
        return self.g[(q, str[0])]

In [48]:
class TSAFM:
    """
    Implementation of a Transition-Assigned Finite State Machine (Mealy Automaton)

    where:
    Q - Set of states
    S - input alphabet
    R - output alphabet
    f - state transition function (Q x S -> Q)
    g - output function (Q x S -> R)
    qi - initial state
    """

    def __init__(self, Q, S, R, f, g, qi):
        self.Q = Q
        self.S = S
        self.R = R
        self.f = f
        self.g = g
        self.q0 = qi

    def run(self, input_str):
        """
        Processes an input string and returns the output generated by the FSM.

        Returns:
        - output: the output generated by the FSM for the input string
        """
        q = self.q0
        output = ""
        for symbol in input_str:
            if symbol not in self.S:
                raise ValueError(f"Invalid input symbol {symbol}")
            q_next = self.f.get((q, symbol), None)
            if q_next is None:
                raise ValueError(f"No transition defined for state {q} and input symbol {symbol}")
            output += self.g.get((q, symbol), "")
            q = q_next
        return output


In [60]:
"""
Sample FSM: Defines a finite state acceptor that accepts binary strings
that contain an odd numbered count of 1's
"""
# Define the parameters for the TSAFM
Q = {'q0', 'q1', 'q2'}
S = {'0', '1'}
R = {'A', 'B'}
f = {('q0', '0'): 'q1', ('q0', '1'): 'q0', ('q1', '0'): 'q1', ('q1', '1'): 'q2', ('q2', '0'): 'q1', ('q2', '1'): 'q0'}
g = {('q0', '0'): 'A', ('q0', '1'): 'B', ('q1', '0'): 'A', ('q1', '1'): 'A', ('q2', '0'): 'B', ('q2', '1'): 'A'}
qi = 'q0'

# Create an instance of the TSAFM
fsm = TSAFM(Q, S, R, f, g, qi)

# Process an input string
input_str = '111'
result = fsm.run(input_str)

# Print  result
print(result)

BBB
